In [29]:
#Dependencies

# from splinter import Browser
# import requests
from webdriver_manager.chrome import ChromeDriverManager

# Using selenium industry standard as suggested
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup
import time

from urllib.parse import urljoin

import pandas as pd


In [2]:
#Urls of page to be scraped

mars_web_site_url = "https://redplanetscience.com/"
mars_images_url = "https://spaceimages-mars.com/"
mars_facts_url = "https://galaxyfacts-mars.com/"
mars_hemisphere_images_site_url = "https://marshemispheres.com/"


In [3]:

# Setup splinter
# executable_path = {'executable_path': ChromeDriverManager().install()}
# browser = Browser('chrome', **executable_path, headless=False)




##### Get page source using Requests/Splinter

In [4]:
# Retrieve page using Requests/Splinter

# browser.visit(mars_web_site_url)
# mars_web_site_splinter_response = browser.html
# mars_web_site_splinter_soup = BeautifulSoup(mars_web_site_splinter_response, 'html.parser')

# news_splinter_title = mars_web_site_splinter_soup.find_all('div', class_='content_title')[0].text.strip()
# news_splinter_p = mars_web_site_splinter_soup.find_all('div', class_='article_teaser_body')[0].text.strip()

#print(news_splinter_title)
#print(news_splinter_p)

##### Get Page source using selenium - Install chrome web driver

In [5]:
# Initialize chrome web driver
driver = webdriver.Chrome(ChromeDriverManager().install()) 



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\jemif\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


##### Get Page source using selenium  - Repetitive code

In [6]:
# driver.get(mars_web_site_url)
# time.sleep(5) 
# mars_web_site_response = driver.page_source

# driver.get(mars_images_url)
# time.sleep(5) 
# mars_images_site_response = driver.page_source

# driver.get(mars_facts_url)
# time.sleep(5) 
# mars_facts_site_response = driver.page_source

# driver.get(mars_hemisphere_images_site_url)
# time.sleep(5) 
# mars_hemispheres_site_url = driver.page_source

##### Get page source - removed repetition by using user defined function

In [45]:
# Retrieve page using Selenium and chrome driver

def get_page_source(url, delay):
    driver.get(url)
    # if dynamic content - better to wait for few seconds for the content to load
    if delay == True:
        time.sleep(5)        
    return driver.page_source


mars_web_site_response = get_page_source(mars_web_site_url, True)
mars_images_site_response = get_page_source(mars_images_url, True)
mars_facts_site_response = get_page_source(mars_facts_url, False)
mars_hemispheres_site_response = get_page_source(mars_hemisphere_images_site_url, False)


##### Create beautifulsoup objects by parsing with 'html.parser'

In [46]:

mars_web_site_soup = BeautifulSoup(mars_web_site_response, 'html.parser')
mars_images_site_soup = BeautifulSoup(mars_images_site_response, 'html.parser')
mars_facts_site_soup = BeautifulSoup(mars_facts_site_response, 'html.parser')
mars_hemispheres_site_soup = BeautifulSoup(mars_hemispheres_site_response, 'html.parser')


##### Scrape the Mars News Site and collect the latest News Title and Paragraph Text. Assign the text to variables for later reference

In [47]:

news_title = mars_web_site_soup.find_all('div', class_='content_title')[0].text.strip()
news_p = mars_web_site_soup.find_all('div', class_='article_teaser_body')[0].text.strip()


##### JPL Mars Space Images - Get current Featured Mars Image

In [48]:
featured_image_url = mars_images_site_soup.find_all('img', class_='headerimage fade-in')[0]['src']
featured_image_url = urljoin(mars_images_url,featured_image_url)


##### Scrape Mars Facts Using Pandas

In [89]:
mars_facts_html = pd.read_html(mars_facts_url)
mars_facts_df = mars_facts_html[0]
new_header = mars_facts_df.iloc[0] #grab the first row for the header
mars_facts_df = mars_facts_df[1:] #take the data less the header row
mars_facts_df.columns = new_header #set the header row as the df header
mars_facts_df.set_index('Mars - Earth Comparison', inplace=True)
mars_facts_df.reset_index()
mars_facts_df

,Mars,Earth
Mars - Earth Comparison,,
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


##### Pandas DataFrame To HTML Table String

In [90]:
html_table_string = mars_facts_df.to_html()
print(html_table_string)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Mars</th>
      <th>Earth</th>
    </tr>
    <tr>
      <th>Mars - Earth Comparison</th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Diameter:</th>
      <td>6,779 km</td>
      <td>12,742 km</td>
    </tr>
    <tr>
      <th>Mass:</th>
      <td>6.39 × 10^23 kg</td>
      <td>5.97 × 10^24 kg</td>
    </tr>
    <tr>
      <th>Moons:</th>
      <td>2</td>
      <td>1</td>
    </tr>
    <tr>
      <th>Distance from Sun:</th>
      <td>227,943,824 km</td>
      <td>149,598,262 km</td>
    </tr>
    <tr>
      <th>Length of Year:</th>
      <td>687 Earth days</td>
      <td>365.24 days</td>
    </tr>
    <tr>
      <th>Temperature:</th>
      <td>-87 to -5 °C</td>
      <td>-88 to 58°C</td>
    </tr>
  </tbody>
</table>


##### Mars Hemispheres

In [75]:
hemisphere_image_urls = []
mars_hemispheres_descriptions = mars_hemispheres_site_soup.find_all('div', class_='description')

for mars_hemispheres_description in mars_hemispheres_descriptions:
    mars_hemispheres_link = mars_hemispheres_description.find("a")
    hemisphere_page_url = urljoin(mars_hemisphere_images_site_url, mars_hemispheres_link['href'])
    title = mars_hemispheres_link.h3.text.strip()
    
    hemisphere_page_html = get_page_source(hemisphere_page_url, False)
    hemisphere_page_soup = BeautifulSoup(hemisphere_page_html, 'html.parser')
    
    downloads_div = hemisphere_page_soup.find("div", class_='downloads')
    
    hemisphere_full_res_img_url = downloads_div.find('ul').find('li', recursive = False).a['href']
    
    hemisphere_image_urls.append(dict({
        "title" : title,
        "img_url" : urljoin(mars_hemisphere_images_site_url ,hemisphere_full_res_img_url)
    }))
    
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg'}]

In [76]:
!jupyter nbconvert --to script --output "scrape_mars" "mission_to_mars.ipynb"

[NbConvertApp] Converting notebook mission_to_mars.ipynb to script
[NbConvertApp] Writing 5022 bytes to scrape_mars.py
